# 📊 Walmart Sales Analysis
This notebook analyzes city-wise sales from a Walmart dataset, visualizing the top cities and overall sales distribution.


import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


file_path = "walmart_sales.csv"
df = pd.read_csv(file_path)
df.head()


In [ ]:
df["unit_price"] = df["unit_price"].replace(r"[\$,]", "", regex=True).astype(float)
df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce").fillna(0)
df["total"] = df["unit_price"] * df["quantity"]
df[["unit_price", "quantity", "total"]].head()


In [ ]:
city_sales = df.groupby("City")["total"].sum()
city_sales = city_sales[city_sales > 0]
city_sales_sorted = city_sales.sort_values(ascending=False)
city_sales_sorted.head(10)


In [ ]:
top_10 = city_sales_sorted[:10]
others = city_sales_sorted[10:].sum()
top_10_with_others = top_10.copy()
top_10_with_others["Others"] = others
top_10_with_others = top_10_with_others.sort_values(ascending=False)
top_10_with_others


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
colors = plt.cm.tab20.colors

# Pie chart
wedges, texts, autotexts = ax1.pie(
    top_10_with_others,
    labels=None,
    autopct="%1.1f%%",
    startangle=140,
    colors=colors,
    textprops={'fontsize': 9}
)
ax1.set_title("Sales Distribution by City", fontsize=14)
ax1.legend(
    top_10_with_others.index,
    title="Cities",
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    fontsize=9
)

# Bar chart
bars = top_10.sort_values().plot.barh(
    ax=ax2, 
    color='cornflowerblue',
    edgecolor='black'
)
ax2.set_xlabel("Total Sales (USD)")
ax2.set_ylabel("City")
ax2.set_title("Top 10 Cities by Total Sales", fontsize=14)
ax2.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))
ax2.bar_label(bars.containers[0], fmt='${:,.0f}', label_type='edge', fontsize=8, padding=3)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()


In [ ]:
output_folder = "Reports"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "city_sales_charts.png")
fig.savefig(output_path, dpi=300, bbox_inches="tight")
print(f"✅ Chart saved to {output_path}")
